In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from PMTK.sampling.preferences_sampler import *
from PMTK.pref.preferences import *
from PMTK.utility.utility_solver import *
from PMTK.sampling.subset_samplers import *
from PMTK.utility.model_solver import *
from PMTK.utility.subset_finder import *
from PMTK.sampling.gibbs import *
from PMTK.data.film_dataset import *
from PMTK.sampling.decider import *
from PMTK.utility.heuristics import * 
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate


In [2]:
n_users = 10
n_films = 50
fd = Film_Dataset(n_films,n_users + 1)
prf = fd.get_preferences_items(n_users + 1)
t_heuristic = build_approx_theta(prf, [EMPTY_SET])

In [3]:
t_mins = get_kernels_lex3(prf,t_heuristic)

In [27]:
class Simple_DM:
    
    def __init__(self, items):
        self.alternatives = []
        self.rates = []
        self.items = items
    
    def rate_alternative(self, alt, rate):
        alt = tuple(alt)
        if alt in self.alternatives:
            i = self.alternatives.index(alt)
            self.alternatives.remove(alt)
            self.rates.remove(self.rates[i])
        self.alternatives.append(alt)
        self.rates.append(rate)
        
    
    def rate_binary_alternative(self, b_alternative, rate):
        alt = tuple(np.where(b_alternative == 1)[0])
        if alt in self.alternatives:
            i = self.alternatives.index(alt)
            self.alternatives.remove(alt)
            self.rates.remove(self.rates[i])
            
        self.alternatives.append(alt)
        self.rates.append(rate)
            
        
    def preferences(self):
        prf = Preferences(self.items)
        for alt_1, r_1 in zip(self.alternatives, self.rates):
            for alt_2, r_2 in zip(self.alternatives, self.rates):
                if r_1 > r_2:
                    prf.add_preference(alt_1, alt_2)
        return prf

In [36]:
def rates_to_preferences(items, alternatives, rates):
    sdm = Simple_DM(items)
    for alt, r in zip(alternatives, rates):
        sdm.rate_binary_alternative(alt, r)
    return sdm.preferences()

def rates_to_binary(rates_matrix, targeted_user, rate_encoder):
    X = np.hstack([rates_matrix[:, :targeted_user], rates_matrix[:, targeted_user+1:]])
    rates = rates_matrix[:, targeted_user]
    encoded_mat = []
    for v in X:
        L = []
        for i in v:
            r_e = rate_encoder(i)
            if r_e != None:
                L = L + rate_encoder(i)
        L = np.array(L)
        encoded_mat.append(L)
    encoded_mat = np.array(encoded_mat)
    pos = np.where(rates != -1)[0]
    rates = rates[pos]
    encoded_mat = encoded_mat[pos, :]
    return encoded_mat, rates

In [29]:
def rates_enc(x):
    if x == -1:
        return [0,0]
    if x <= 3 and x >= 2:
        return [0,0]
    if x > 3:
        return [1,1]
    if x < 2:
        return [1,0]
    return None

In [43]:
def get_rates_matrix(f_path, targeted_user, bound_films = 50, bound_users = 5):
    df = pd.read_csv(f_path)
    
    u_df = pd.read_csv(f_path)
    u_df = u_df[u_df.rating > 0].groupby("userId").count().reset_index()
    count_u = {u:m for u,m in zip(u_df["userId"], u_df["movieId"])}
    count_u = {i:j for i,j in sorted(count_u.items(), key = lambda x:x[1], reverse= True)}
    targeted_id = list(count_u.keys())[targeted_user]
    
    
    target_df = df[df.userId == targeted_id]

    
    m_df = df[df.movieId.isin(target_df.movieId)].groupby("movieId").count().reset_index()
    count_m = {m:u for u,m in zip(m_df["userId"], m_df["movieId"])}
    count_m = {i:j for i,j in sorted(count_m.items(), key = lambda x:x[1], reverse= True)}
    films_id = list(count_m.keys())[:bound_films]
    #print(count_m)
    
    user_df = df[df.movieId.isin(films_id)]
    user_df = user_df.groupby("userId").count().reset_index()
    count_u = {u:m for u,m in zip(user_df["userId"], user_df["movieId"])}
    count_u = {i:j for i,j in sorted(count_u.items(), key = lambda x:x[1], reverse= True)}
    users_id = list(count_u.keys())[:bound_users]
    if targeted_id in users_id:
        users_id = list(count_u.keys())[:bound_users+1]
        users_id.remove(targeted_id)
    users_id = [targeted_id] + users_id
    
    print(users_id)
    print(films_id)

    
    rates_matrix = np.zeros((len(films_id), len(users_id))) + 188
    for u in users_id:
        for f in films_id:
            x = df[(df.movieId == f) & (df.userId == u)]
            if x.shape[0] == 0:
                rates_matrix[films_id.index(f), users_id.index(u)] = -1
            else:
                rates_matrix[films_id.index(f), users_id.index(u)] = x["rating"].values[0]
    
                
    return rates_matrix

def matrix_to_df(mat):
    user_ids = np.arange(mat.shape[1])
    movie_ids = np.arange(mat.shape[0])
    data = {
        "movieId":[],
        "userId":[],
        "rating":[]
    }
    for u in user_ids:
        for m in movie_ids:
            data["movieId"].append(m)
            data["userId"].append(u)
            data["rating"].append(mat[m,u])
    df = pd.DataFrame(data)
    return df
            

In [32]:
rates_mat = get_rates_matrix("data/ratings.csv", 1)

[599, 414, 68, 274, 483, 590]
[356, 318, 296, 593, 2571, 260, 480, 110, 589, 2959, 1, 1196, 50, 2858, 47, 780, 150, 1198, 4993, 1210, 858, 457, 592, 2028, 5952, 7153, 588, 608, 2762, 380, 32, 364, 377, 1270, 3578, 4306, 1580, 648, 344, 4226, 367, 6539, 58559, 595, 1214, 1036, 165, 500, 1265, 79132]


In [34]:
b_mat, rates = rates_to_binary(rates_mat, 0, rates_enc)


In [37]:
p = rates_to_preferences(np.arange(b_mat.shape[1]), b_mat, rates)

In [ ]:
def train_dec_knn_model(decomposition, knn):
    

In [44]:
from sklearn.decomposition import TruncatedSVD

In [46]:
movie_svd = TruncatedSVD(n_components=3)
movie_features = movie_svd.fit_transform(rates_mat)

In [47]:
movie_features

array([[ 1.08710039e+01,  7.13788568e-01, -5.33019037e-01],
       [ 1.06645202e+01,  1.24834084e+00, -3.82488497e-01],
       [ 1.08620099e+01,  2.28170302e+00, -5.90609651e-01],
       [ 6.97906650e+00, -2.24381750e+00,  8.41623852e-02],
       [ 1.10155090e+01, -1.33636685e-01,  2.66607452e-01],
       [ 1.12171113e+01, -1.01358253e+00, -7.84114546e-01],
       [ 9.38372873e+00,  1.12655737e-01, -2.56901543e-01],
       [ 9.67278368e+00,  1.18294775e+00, -2.46860224e-01],
       [ 1.06438482e+01,  4.06666494e-01, -2.74765868e-01],
       [ 1.04422927e+01,  1.82696310e+00,  3.34102958e-01],
       [ 8.81807196e+00,  9.46543987e-01, -3.59731508e-01],
       [ 1.18181870e+01, -5.60713479e-01, -4.09814945e-02],
       [ 1.00715352e+01,  7.97411808e-01, -6.37650648e-01],
       [ 1.09965089e+01, -2.86463900e-01,  1.68674210e+00],
       [ 9.16437010e+00, -1.91127829e-01,  9.15522084e-01],
       [ 8.98148349e+00, -8.28755572e-01, -9.65333189e-02],
       [ 6.21361061e+00, -2.09798858e+00